In [ ]:
!unzip -u '/content/drive/MyDrive/recommendation sys data/data.zip' -d '/content/drive/MyDrive/recommendation sys data'

Archive:  /content/drive/MyDrive/recommendation sys data/data.zip
   creating: /content/drive/MyDrive/recommendation sys data/data/
  inflating: /content/drive/MyDrive/recommendation sys data/data/test.csv  
  inflating: /content/drive/MyDrive/recommendation sys data/data/train.csv  


In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095455 sha256=95d55201ce65bc014814f33b2141d3f284b22d6295a48ad8c5c394784f0e31f3
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import SVDpp
from surprise import KNNBasic
from surprise.model_selection import cross_validate

train_path = '/content/drive/MyDrive/recommendation sys data/data/train.csv'
test_path = '/content/drive/MyDrive/recommendation sys data/data/test.csv'

df = pd.read_csv(train_path)
df.head()

,userId,itemId,rating,date
0,cgexjc,682978,4,2017-08-13 13:23:35
1,cgexjc,320898,4,2019-01-18 15:56:07
2,cgexjc,29028,4,2017-08-13 14:03:55
3,cgexjc,399148,5,2017-08-13 13:59:51
4,cgexjc,734055,4,2019-01-18 15:37:29


In [4]:
# We Must change the userId column with numeric
def FeatureEncoder(value):
    encoder = {}
    for val, key in enumerate(value):
        encoder[key] = val
    return encoder

dicts = FeatureEncoder(df['userId'])
df['userId'] = df['userId'].map(dicts)
df.tail()

,userId,itemId,rating,date
8612102,8612106,148210,1,2017-08-05 14:15:09
8612103,8612106,735535,4,2017-08-05 14:14:35
8612104,8612106,124242,3,2017-08-05 14:25:33
8612105,8612106,948393,4,2017-08-05 13:25:21
8612106,8612106,163356,4,2017-08-05 14:24:57


In [5]:
df["rating"].unique()

array([4, 5, 3, 2, 1])

In [6]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['userId', 'itemId', 'rating']], reader)

In [7]:
algo = SVDpp(n_epochs=5, n_factors=10)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=2, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    0.8531  0.8530  0.8531  0.0000  
MAE (testset)     0.6550  0.6551  0.6551  0.0000  
Fit time          463.62  542.43  503.02  39.41   
Test time         1844.48 2161.76 2003.12 158.64  


{'test_rmse': array([0.85308721, 0.85304464]),
 'test_mae': array([0.65503111, 0.6551179 ]),
 'fit_time': (463.6190962791443, 542.4298968315125),
 'test_time': (1844.4833459854126, 2161.7611079216003)}

In [8]:
algo.predict(uid=8612106, iid=735535, r_ui=4)

Prediction(uid=8612106, iid=735535, r_ui=4, est=3.125150740801887, details={'was_impossible': False})

In [9]:
for user, item, rating in zip(df['userId'][-30:], df['itemId'][-30:], df['rating'][-30:]):
  pred = algo.predict(uid=user, iid=item, r_ui=rating)
  print(pred)

user: 8612106    item: 64326      r_ui = 4.00   est = 3.31   {'was_impossible': False}
user: 8612106    item: 728049     r_ui = 5.00   est = 3.44   {'was_impossible': False}
user: 8612106    item: 877907     r_ui = 3.00   est = 3.50   {'was_impossible': False}
user: 8612106    item: 541168     r_ui = 4.00   est = 2.99   {'was_impossible': False}
user: 8612106    item: 754693     r_ui = 4.00   est = 3.46   {'was_impossible': False}
user: 8612106    item: 502836     r_ui = 1.00   est = 2.61   {'was_impossible': False}
user: 8612106    item: 766197     r_ui = 5.00   est = 3.45   {'was_impossible': False}
user: 8612106    item: 430639     r_ui = 3.00   est = 3.44   {'was_impossible': False}
user: 8612106    item: 802865     r_ui = 5.00   est = 3.58   {'was_impossible': False}
user: 8612106    item: 744666     r_ui = 1.00   est = 3.12   {'was_impossible': False}
user: 8612106    item: 670107     r_ui = 1.00   est = 2.36   {'was_impossible': False}
user: 8612106    item: 774520     r_ui = 1.

In [10]:
df_res = pd.DataFrame(columns=['prediction'])
df_res

,prediction


In [11]:
df_test = pd.read_csv(test_path)
dicts = FeatureEncoder(df_test['userId'])
df_test['userId'] = df_test['userId'].map(dicts)
df_test.tail()

,userId,itemId,date
131353,131357,388934,2021-07-30 08:48:06
131354,131357,571188,2021-07-30 08:49:17
131355,131357,707682,2021-07-30 08:49:40
131356,131357,44761,2021-07-30 08:48:55
131357,131357,699794,2021-07-30 14:21:08


In [12]:
rating_pred = []
for user, item in zip(df_test['userId'], df_test['itemId']):
  pred = algo.predict(uid=user, iid=item)
  rating_pred.append(pred[-2])

df_res['prediction'] = rating_pred

In [13]:
df_res

,prediction
0,3.829348
1,3.770949
2,3.668109
3,3.317591
4,3.611890
...,...
131353,4.100891
131354,3.524174
131355,3.496714
131356,2.925840


In [14]:
df_res.to_csv('output.csv')
!cp output.csv "drive/My Drive/"